# Computational Theory Problems

## Problem 1: Binary Words and Operations

In [1]:
import numpy as np

# Configure numpy to work with 32-bit unsigned integers
UINT32 = np.uint32

def Parity(x, y, z):
    """
    Parity function: (x ⊕ y ⊕ z)
    
    Reference: SHS Standard (FIPS PUB 180-4), Section 4.1.1 (page 10)
    This section defines how Parity is used in the SHA-1 algorithm.
    The Parity function is specified as x XOR y XOR z.
    Used in: SHA-1 algorithm (rounds 20-39 and 60-79)
    
    Computes the bitwise XOR on each bit of three 32-bit words.
    Examples: returns 1 if an odd number of inputs are 1, else returns 0.
    x == 1 y == 1 z == 0 result == 0
    x == 1 y == 0 z == 1 result == 0
    x == 0 y == 1 z == 1 result == 0 
    x == 1 y == 1 z == 1 result == 1

    Parity = 0b0001 = 0b1
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result of x XOR y XOR z
    
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32(x ^ y ^ z)

In [2]:
# Example test for the Parity function
x, y, z = UINT32(0b1101), UINT32(0b1011), UINT32(0b0111)

print("x:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Parity:", bin(int(Parity(x, y, z)))) # Expected output: 0b1 (result of x XOR y XOR z)


x: 0b1101
y: 0b1011
z: 0b111
Parity: 0b1


In [3]:
def Ch(x, y, z):
    """
    Choose function: (x ∧ y) ⊕ (¬x ∧ z)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    This section defines how the Choose function is used in the SHA-256 algorithm.
    The Choose function selects bits from y and z based on the bits of x.
    Used in: SHA-256 compression function
    
    For each bit position:
    - If x bit is 1, choose the corresponding bit from y
    - If x bit is 0, choose the corresponding bit from z
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result where x "chooses" between y and z
    
    Example:
        x = 1100 (chooses y for first 2 bits, z for last 2 bits)
        y = 1010
        z = 0111
        --------
        Result = 1011 (takes 10 from y, 11 from z)
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32((x & y) ^ (~x & z))

In [4]:
# Example test for the Ch function
x, y, z = UINT32(0b1100), UINT32(0b1010), UINT32(0b0111)

print("\nx:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Ch:", bin(int(Ch(x, y, z)))) # Expected output: 0b1011


x: 0b1100
y: 0b1010
z: 0b111
Ch: 0b1011


In [ ]:
def Maj(x, y, z):
    """
    Majority function: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    This section defines how the Majority function is used in the SHA-256 algorithm.
    The Majority function returns the majority bit for each bit position among x, y, and z
    Used in: SHA-256 compression function
    
    For each bit position:
    - Returns 1 if at least 2 of the 3 bits are 1
    - Returns 0 if at least 2 of the 3 bits are 0
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result representing the majority vote of each bit position
    
    Example:
        x = 1100
        y = 1010
        z = 1001
        --------
        Bit 0: 1,1,1 → majority is 1
        Bit 1: 1,0,0 → majority is 0
        Bit 2: 0,1,0 → majority is 0
        Bit 3: 0,0,1 → majority is 0
        Result = 1000
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32((x & y) ^ (x & z) ^ (y & z))

In [6]:
x, y, z = UINT32(0b1100), UINT32(0b1010), UINT32(0b1001)

print("\nx:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Maj:", bin(int(Maj(x, y, z)))) # Expected output: 0b1000


x: 0b1100
y: 0b1010
z: 0b1001
Maj: 0b1000


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords